# Learning 1d Pure Advection in Conservative Form

To verify that our PINN implementation is working well, we test the PINN on the following problem:
$$
    \begin{cases}
        u_t + a(t)u_x = 0\\
        \lim_{|x|\rightarrow\infty}u(t, x) = 0\\
        u(0, x) = u_0(x)
    \end{cases}
$$ where $u_0 = \mathcal{N}(x|0, 1)$. 

In this case, the NN solution is assumed to have the following model (conservative advection):
$$
    \frac{\partial u_{\theta_1}}{\partial t} + \frac{\partial}{\partial x}\bigg(\mathcal{V}_{\theta_2}(t)u_{\theta_1}\bigg) = 0
$$

In this test case, the true advection velocity is a time-varying function, no spatial dependence. Furthermore, since the coefficient neural network is a time-dependent function, the conservative and non-conservative form of the PDE coincide. In this notebook, we generate data from such a PDE and test the PINN learning in both conservative mode and non-conservative mode.

In this note, we use the Fourier embedded deep neural nets to overcome the high frequency in time domain.

In [ ]:
from PINN.AdvectionNet import *
from PINN.utils.dnn import *
# Testing
import matplotlib.pyplot as plt
import torch
import numpy as np
import scipy

# set random seeds
np.random.seed(10)
torch.manual_seed(10);

import numpy as np    
import matplotlib.pyplot as plt

import time
import pylab as pl
from IPython import display
from IPython.display import clear_output

# Experiment 1: High Frequency with Vanilla NN

Consider the following PDE, taken from: https://scullen.com.au/DSc/Publications/scullen_92.pdf (Page 44)

$$
    \frac{\partial u}{\partial t} + v_0\sin(\omega t)\frac{\partial u}{\partial x} = 0
$$ which admits an analytical solution:
$$
    u(t,x) = \exp\bigg(
        -k[x - x_0 - 2v_0\omega^{-1}\sin^2(\frac{1}{2}\omega t)]^2
    \bigg)
$$ with parameters $k, x_0$ from the initial condition.

In [ ]:
# parameters
x0 = 2.0
v0 = 2.0
k = 5.0
omega = 2.0*np.pi
# time grid
t_start = 0.0
t_end = 2*np.pi
dt = 0.001
tgrid = np.arange(t_start, t_end, dt)
nt = len(tgrid)
# spatial grid
x_left, x_right = 0.0, 5.0
dx = 0.005
xgrid = np.arange(x_left, x_right, dx)
nx = len(xgrid)

# solution
u_sol = np.zeros([nt, nx])
for i in range(nt):
    t = tgrid[i]
    u_sol[i, :] = np.exp(-k * (( xgrid - x0 ) - (2*v0/omega) * (np.sin(0.5*omega*t) ** 2)) ** 2 )

In [ ]:
# plot velocity as a function of time
plt.figure(2);
plt.plot(tgrid, v0*np.sin(omega*tgrid), "--", color="blue", alpha=0.6, lw=4.0);
plt.xlabel(r"$t$"); plt.ylabel(r"Velocity (High Frequency)");

In [ ]:
# save data for PINN training
scipy.io.savemat("../data/LinearOscillator/Time_Variable_Advection.mat", {
    "xgrid": xgrid.reshape(1, -1),
    "tgrid": tgrid.reshape(1, -1),
    "pmc": u_sol
})

# save a smaller dataset
subsample_t = 10
subsample_x = 5
xgrid_small = xgrid.reshape(1, -1)[:, 0:-1:subsample_x].squeeze(),
tgrid_small = tgrid.reshape(1, -1)[:, 0:-1:subsample_t].squeeze()
u_sol_small = u_sol[0:-1:subsample_t, 0:-1:subsample_x]
scipy.io.savemat("../data/LinearOscillator/Time_Variable_Advection_small.mat", {
    "xgrid": xgrid_small,
    "tgrid": tgrid_small,
    "pmc": u_sol_small
})
print("Training data size = {}\n".format(u_sol_small.shape))

In [ ]:
plt.figure(1, figsize=(10, 10));
plt.imshow(u_sol_small);
plt.title("Training Data");

Directly train the net on solution as a regression problem (data loss only)

In [ ]:
# create training data from grids
X = cartesian_data(torch.tensor(tgrid_small.flatten()), torch.tensor(xgrid_small[0].flatten()))
y = torch.tensor(u_sol_small).T.flatten().reshape(-1, 1)
def train(inputs, outputs, model, optim, scheduler, batch_size, epochs, shuffle=True):
    X, y = inputs, outputs
    nx = X.shape[0]
    num_batches = int(nx/batch_size)
    for i in range(epochs):
        print("============================================================\n")
        print("Epoch = {}\n".format(i+1));
        print("============================================================\n")
        model.train()
        if shuffle:
            tmp = np.random.permutation(nx)
            X, y = X[tmp, :].data.clone(), y[tmp, :].data.clone()
        for idx in range(num_batches):
            if idx % 100 == 0:
                print("| => | Batch {} |\n".format(idx+1))
        # closure definition
            def closure():
                optim.zero_grad()
                start_idx = idx*batch_size
                end_idx = (idx+1)*batch_size
                if idx + 1 == num_batches:
                    # if last batch
                    end_idx = -1
                Xb, yb = X[start_idx:end_idx, :].data.clone(), y[start_idx:end_idx, :].data.clone()

                # require gradients
                Xb.requires_grad = True
                # make a prediction on the batch
                y_pred = model.forward(Xb)
                # compute L^2 loss
                loss = torch.mean((y_pred - yb)**2)
                # backpropagate
                loss.backward()
                if idx % 100 == 0:
                    print("==> Batch {} loss = {}".format(idx, loss.item()))
                return loss
            optim.step(closure=closure)
        if scheduler:
            # step scheduler after epoch if there is one
            scheduler.step()
            print("---------- \n")
            print("++ Learning rate reduced, now at = {}".format(scheduler.get_last_lr()[0]))

In [ ]:
nn_fourier2d_cartesian = FourierProductEmbeddedDNN2d(
    layers_time=[30, 200, 200, 1], 
    layers_space=[30, 200, 200, 1], 
    activation=torch.nn.Tanh, 
    last_layer_activation=None, 
    mt=15, 
    mx=15, 
    freq_stds_t=[1.,2., 5., 10.], 
    freq_stds_x=[1., 2., 5., 10.]
)
optim = torch.optim.Adam(
    nn_fourier2d_cartesian.parameters(),
    lr=8e-3
)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.9999)
train(X, y, nn_fourier2d_cartesian, optim, scheduler, 2**9, 50, shuffle=True)

In [ ]:
# save model
torch.save(nn_fourier2d_cartesian.state_dict(), "./PINN/models/Fourier2dNet_Time15_Space15")

In [ ]:
# predict
nx_small = len(xgrid_small[0])
nt_small = len(tgrid_small)
u_sol_predict_fourier = nn_fourier2d_cartesian(X).reshape([nx_small, nt_small]).detach().numpy().T
u_sol_exact_fourier = y.reshape([nx_small, nt_small]).detach().numpy().T

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 10))
ax[0].imshow(u_sol_predict_fourier)
ax[1].imshow(u_sol_exact_fourier)
cax = ax[2].imshow(np.abs(u_sol_exact_fourier-u_sol_predict_fourier));
fig.colorbar(cax, orientation='horizontal');

PINN experiment

In [ ]:
# PINN experiments
data_path = "../data/LinearOscillator/Time_Variable_Advection_small.mat"
# create PINN
pinn = AdvectionNet(indim=2, outdim=1, data_path=data_path, scheduler="ExponentialLR", coef_mode="time")

In [ ]:
# reinitialize advection coefficient to Fourier-based net
pinn.G_nn = FourierEmbeddedDNN(
    layers=[30, 50, 50, 1],
    activation=torch.nn.Tanh,
    last_layer_activation=None,
    initialization=None,
    m=15,
    freq_stds=[1, 2, 5, 10]
)
pinn.G_nn.mode = "time"

# reinitialize solution net to Fourier-based net 
pinn.p_nn = FourierProductEmbeddedDNN2d(
    layers_time=[30, 100, 100, 1], 
    layers_space=[30, 50, 50, 1], 
    activation=torch.nn.Tanh, 
    last_layer_activation=None, 
    mt=15, 
    mx=15, 
    freq_stds_t=[1.,2., 10., 20., 100.], 
    freq_stds_x=[1., 2., 5., 10.]
)

In [ ]:
# testing training
info = train_pinn(
    pinn, pinn.optimizer, pinn.scheduler, batch_size=2**9,
    epochs=200, batch_print=200, mode="all", conservative_pde=True
)

In [ ]:
inputs = cartesian_data(pinn.tgrid, pinn.xgrid)
G_pred = pinn.G_nn(inputs[:, 0][:, None]).reshape([pinn.nx, pinn.nt]).detach().numpy().T
# spatial and temporal grid
tgrid = pinn.tgrid.detach().numpy()
xgrid = pinn.xgrid.detach().numpy()

In [ ]:
plt.plot(tgrid, G_pred[:, 0], color="red", lw=2.0, label="predicted");
plt.plot(tgrid, v0*np.sin(omega*tgrid), "--", color="blue", alpha=0.6, lw=4.0);
plt.title("Inferred advection coefficient as a function of time (Fourier Net)");
plt.legend();